In [1]:
import os
from pathlib import Path
import librosa
import re
from time import time
import ast
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from IPython.display import Audio, display
import soundfile as sf
from scipy.io import wavfile

from src.utils.SpectrogramExtractor import SpectrogramExtractor         # Custom class to create a spectrogram from a .wav

In [59]:
def plot_spectrograms_from_paths(spec_paths, df, hop_ms=10):
    """
    Vertically stacked spectrograms with shared x-axis. Only bottom plot shows axis labels.

    Args:
        spec_paths (list): List of full paths to spectrogram .pt files.
        df (pd.DataFrame): DataFrame with 'spec_dir' and 'speaker_count' columns.
        hop_ms (int): Hop size in milliseconds for time axis calculation.
    """
    n_clips = len(spec_paths)
    fig, axes = plt.subplots(n_clips, 1, figsize=(5, 1.75 * n_clips), sharex=True)

    for idx, (spec_path, ax) in enumerate(zip(spec_paths, axes)):
        spectrogram = torch.load(spec_path).squeeze()

        # Lookup speaker count
        match = df.loc[df['spec_dir'] == spec_path, 'speaker_count']
        speaker_count = match.values[0] if not match.empty else 'Unknown'

        # Time axis
        n_frames = spectrogram.shape[1]
        duration_sec = (n_frames * hop_ms) / 1000

        # Plot
        im = ax.imshow(spectrogram.numpy(), aspect='auto', origin='lower',
                       extent=[0, duration_sec, 0, spectrogram.shape[0]],
                       cmap='magma')

        ax.set_title(f'Speaker Count: {speaker_count}', fontsize=11)

        # Y-axis cleanup
        ax.set_ylabel(
        "(Hz)",    
        rotation=0,            
        labelpad=12,
        va="center" 
        )

        ax.set_yticks([])

        # X-axis formatting
        if idx < n_clips - 1:
            ax.tick_params(axis='x', labelbottom=False)  # Show ticks, hide labels
        else:
            ax.set_xlabel('Time (s)')

    fig.tight_layout()
    cbar = fig.colorbar(im, ax=axes, location='right', shrink=0.6)

    # Set custom ticks: first, middle, last
    cbar_ticks = [im.norm.vmin, (im.norm.vmin + im.norm.vmax) / 2, im.norm.vmax]
    cbar.set_ticks(cbar_ticks)
    cbar.set_ticklabels(['-', '0', '+'])

    # Optional: label
    cbar.set_label('Amplitude', rotation=270, labelpad=15)

    plt.show()

In [2]:
spec_paths = [
     r'data\specs_rir\largeroom1_1_clip1.pt',
     r'data\specs_libricss\0L_session0_clip28.pt',
    r'data\specs_aug\overlay_1.pt',
    r'data\specs_aug\overlay_25500.pt',
    r'data\specs_aug\overlay_52500.pt'
]

plot_spectrograms_from_paths(spec_paths, df)

NameError: name 'plot_spectrograms_from_paths' is not defined